# Check Requiments for office location election:

In [5]:
import pandas as pd
import requests
import os
from dotenv import load_dotenv
from pymongo import MongoClient
from pandas.io.json import json_normalize
import numpy as np
from API import *

## 1º Requirement: Developers like to be near successful tech startups that have raised at least 1 Million dollars.

In [6]:
dbName = "companies"
mongodbURL = f"mongodb://localhost/{dbName}"
print(mongodbURL)
client = MongoClient(mongodbURL, connectTimeoutMS=2000,serverSelectionTimeoutMS=2000)
db = client.get_database()

mongodb://localhost/companies


**Importamos dataset con las  tech companies que tengan offices en Seattle:**

In [7]:
query={"$and":[{"offices.city":{"$eq":"Seattle"}},{"category_code":{"$in":["software","games_video","web","network_hosting","cleantech","biotech","nanotech"]}}]}
data=list(db.companies.find(query,{"name":1,"category_code":1,"total_money_raised":1,"offices":1}))
data_companies_seattle = pd.DataFrame(data)
data_companies_seattle=data_companies_seattle.explode("offices")
data_companies_seattle.head()

,_id,name,category_code,total_money_raised,offices
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave..."
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'description': '', 'address1': '270 Lafayette..."
1,52cdef7c4bab8bd675297dfc,TripHub,web,$0,"{'description': None, 'address1': '2033 6th Av..."
2,52cdef7c4bab8bd675297e0b,Newsvine,web,$1.25M,"{'description': None, 'address1': '101 Elliott..."
3,52cdef7c4bab8bd675297e11,iLike,games_video,$16.5M,"{'description': None, 'address1': '1605 Boylst..."


**Seleccionamos solo las companies que tengan un "total money raised" mayor a 1 millón:**

In [8]:
# limpiar todo lo que sea menor a 1 million raised
money_raised_lessmillion=["$0","$500k"]
data_companies_seattle["million_money_raised"]=np.where(data_companies_seattle["total_money_raised"].isin(money_raised_lessmillion),1,2)
data_companies_seattle.drop(data_companies_seattle[data_companies_seattle["million_money_raised"]<2].index,inplace=True)
data_companies_seattle.head()

,_id,name,category_code,total_money_raised,offices,million_money_raised
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'description': '', 'address1': '710 - 2nd Ave...",2
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'description': '', 'address1': '270 Lafayette...",2
2,52cdef7c4bab8bd675297e0b,Newsvine,web,$1.25M,"{'description': None, 'address1': '101 Elliott...",2
3,52cdef7c4bab8bd675297e11,iLike,games_video,$16.5M,"{'description': None, 'address1': '1605 Boylst...",2
4,52cdef7c4bab8bd675297e6d,PayScale,software,$31.1M,"{'description': '', 'address1': '542 First Ave...",2


**Creación columna *location* con información de latitud y longitud de la columna *offices* :**

In [9]:
def officeToGeoPoint(row):
    office = row.offices
    if type(office) == dict:
        if 'latitude' in office and 'longitude' in office:
            if(type(office["latitude"])) == float and type(office["longitude"]) == float:
                return ({
                    "type":"Point",
                    "coordinates":[office["longitude"],office["latitude"]]
                },"success")
            else:
                return(None,"Invalid lat lat and long")
        else:
            return (None,"No lat and long keys in office dict")
    return (None,"No office")

In [10]:
offices = data_companies_seattle.apply(officeToGeoPoint,axis=1, result_type="expand")
offices.columns=["office","clean_state"]
offices.head()

,office,clean_state
0,"{'type': 'Point', 'coordinates': [-122.333253,...",success
0,"{'type': 'Point', 'coordinates': [-73.9964312,...",success
2,"{'type': 'Point', 'coordinates': [-122.358484,...",success
3,"{'type': 'Point', 'coordinates': [-122.323408,...",success
4,"{'type': 'Point', 'coordinates': [-122.3276962...",success


*Juntamos el dataframe creado con las nuevas columnas al dataframe original con las oficinas en Seattle; eliminamos filas nulas y columnas sobrantes*

In [11]:
offices_seattle_data=pd.concat([data_companies_seattle,offices],axis=1)
offices_seattle_data=offices_seattle_data.dropna(subset=["office"])
offices_seattle_data=offices_seattle_data.drop(columns=["million_money_raised","offices","clean_state"])
offices_seattle_data.shape

(56, 5)

**Creamos columnas "Latitude" y "Longitude" para separar contenido columna "office":**

In [12]:
def easyLatLng(row):
    of= row['office']
    return {
        "latitude":of["coordinates"][1],
        "longitude":of["coordinates"][0]
    }
offices_seattle_data = pd.concat([offices_seattle_data, offices_seattle_data.apply(easyLatLng, axis=1, result_type="expand")],axis=1)
offices_seattle_data.head()

,_id,name,category_code,total_money_raised,office,latitude,longitude
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'type': 'Point', 'coordinates': [-122.333253,...",47.603122,-122.333253
0,52cdef7c4bab8bd675297d8a,Wetpaint,web,$39.8M,"{'type': 'Point', 'coordinates': [-73.9964312,...",40.723731,-73.996431
2,52cdef7c4bab8bd675297e0b,Newsvine,web,$1.25M,"{'type': 'Point', 'coordinates': [-122.358484,...",47.618599,-122.358484
3,52cdef7c4bab8bd675297e11,iLike,games_video,$16.5M,"{'type': 'Point', 'coordinates': [-122.323408,...",47.615313,-122.323408
4,52cdef7c4bab8bd675297e6d,PayScale,software,$31.1M,"{'type': 'Point', 'coordinates': [-122.3276962...",47.599934,-122.327696


In [13]:
offices_seattle_data=offices_seattle_data.drop("_id", axis=1)
offices_seattle_data.to_json("OUTPUT/offices_seattle.json",orient="records")

**Creación nuevo dataset con las compañias que cumplen el 1º requirement:**

In [14]:
millioncompanies_data=offices_seattle_data[["name","latitude","longitude","office"]]
millioncompanies_data["Requirement"]=np.where(millioncompanies_data["name"].isnull(),"","TechCompanies raised more 1 million")
millioncompanies_data=millioncompanies_data.rename(columns={"office":"location"})
column_order=["Requirement","name","latitude","longitude","location"]
millioncompanies_data=millioncompanies_data[column_order]
millioncompanies_data.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Requirement,name,latitude,longitude,location
0,TechCompanies raised more 1 million,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,TechCompanies raised more 1 million,Wetpaint,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,TechCompanies raised more 1 million,Newsvine,47.618599,-122.358484,"{'type': 'Point', 'coordinates': [-122.358484,..."
3,TechCompanies raised more 1 million,iLike,47.615313,-122.323408,"{'type': 'Point', 'coordinates': [-122.323408,..."
4,TechCompanies raised more 1 million,PayScale,47.599934,-122.327696,"{'type': 'Point', 'coordinates': [-122.3276962..."


##  GEOCODE , FOURSQUARE  y GOOGLEPLACES API:

**Vamos a utilizar estas apis para buscar el contenido del resto de requirements:**

**Utilizo Geocode para encontrar coordenadas de un punto centrico en Seattle (Seattle Downtown):**

In [25]:
seattle = "Seattle Downtonwn"
geocode(seattle)

WE HAVE APIKEY


{'coordinates': [-122.33279, 47.62405]}

## 2º Requirement: The CEO is Vegan

**Buscamos restaurantes veganos de la zona:**

In [26]:
load_dotenv()
vegan_result=getFromFoursquare('47.61186,-122.33581',5000,"Vegan Restaurant","4bf58dd8d48988d1d3941735")
Vegan_restaurants=[]
for resp in vegan_result["response"]["groups"]:
    for item in resp["items"]:
        vegan_name=item['venue']["name"]
        vegan_latitud=item['venue']["location"]["lat"]
        vegan_longitud=item['venue']["location"]["lng"]
        Vegan_restaurants.append({"Requirement":"Vegan Restaurant","name":vegan_name,"latitude":vegan_latitud,"longitude":vegan_longitud,"location":{"type":"Point","coordinates":[vegan_longitud,vegan_latitud]}})
          
Vegan_data=pd.DataFrame(Vegan_restaurants)
Vegan_data.head()

,Requirement,name,latitude,longitude,location
0,Vegan Restaurant,Veggie Grill,47.609882,-122.336637,"{'type': 'Point', 'coordinates': [-122.3366367..."
1,Vegan Restaurant,Plum Bistro,47.613801,-122.316933,"{'type': 'Point', 'coordinates': [-122.3169331..."
2,Vegan Restaurant,Veggie Grill,47.623078,-122.336920,"{'type': 'Point', 'coordinates': [-122.3369198..."
3,Vegan Restaurant,Kati Vegan Thai,47.620864,-122.333340,"{'type': 'Point', 'coordinates': [-122.3333401..."
4,Vegan Restaurant,Pantry by Plum,47.621221,-122.350860,"{'type': 'Point', 'coordinates': [-122.3508600..."


In [27]:
Vegan_data.to_json("OUTPUT/vegan_restaurants.json",orient="records")

## 3º Requirement: Account managers need to travel a lot

**Para este requirement, vamos a buscar los metros, estaciones de tren (Foursquare) y aeropuertos (GoogleApi):**

In [28]:
metro_result=getFromFoursquare('47.60312,-122.32384',5000,"Metro","4bf58dd8d48988d1fd931735")
metro_list=[]
for resp in metro_result["response"]["groups"]:
    for item in resp["items"]:
        metro_name=item['venue']["name"]
        metro_latitud=item['venue']["location"]["lat"]
        metro_longitud=item['venue']["location"]["lng"]
        metro_list.append({"Requirement":"Transport - Metro","name":metro_name,"latitude":metro_latitud,"longitude":metro_longitud,"location":{"type":"Point","coordinates":[metro_longitud,metro_latitud]}})
metro_data=pd.DataFrame(metro_list)  
metro_data.head()

,Requirement,name,latitude,longitude,location
0,Transport - Metro,Metro Stop #1080,47.613965,-122.332277,"{'type': 'Point', 'coordinates': [-122.332277,..."
1,Transport - Metro,International District/Chinatown LINK Station,47.598593,-122.328000,"{'type': 'Point', 'coordinates': [-122.3280000..."
2,Transport - Metro,Pioneer Square LINK Station,47.602276,-122.330999,"{'type': 'Point', 'coordinates': [-122.3309994..."
3,Transport - Metro,King County Metro Transit HQ,47.599115,-122.331284,"{'type': 'Point', 'coordinates': [-122.3312838..."
4,Transport - Metro,King County Metro Route 131,47.606642,-122.335198,"{'type': 'Point', 'coordinates': [-122.3351977..."


In [29]:
train_result=getFromFoursquare('47.60312,-122.32384',10000,"Train Station","4bf58dd8d48988d129951735")
trains_list=[]
for resp in train_result["response"]["groups"]:
    for item in resp["items"]:
        train_name=item['venue']["name"]
        train_latitud=item['venue']["location"]["lat"]
        train_longitud=item['venue']["location"]["lng"]
        trains_list.append({"Requirement":"Transport - Train","name":train_name,"latitude":train_latitud,"longitude":train_longitud,"location":{"type":"Point","coordinates":[train_longitud,train_latitud]}})

trainstation_data=pd.DataFrame(trains_list)
trainstation_data.head()

,Requirement,name,latitude,longitude,location
0,Transport - Train,King Street Station (SEA),47.598165,-122.330057,"{'type': 'Point', 'coordinates': [-122.3300572..."
1,Transport - Train,605 Union Station,47.603813,-122.329789,"{'type': 'Point', 'coordinates': [-122.3297889..."
2,Transport - Train,King Street Gate 7,47.598393,-122.329291,"{'type': 'Point', 'coordinates': [-122.329291,..."
3,Transport - Train,Amtrak Empire Builder,47.597992,-122.329985,"{'type': 'Point', 'coordinates': [-122.3299852..."
4,Transport - Train,Track 7 King Street Station,47.597633,-122.331290,"{'type': 'Point', 'coordinates': [-122.33129, ..."


In [30]:
queryParams = {
    "location": "47.61186, -122.33581",
    "radius": "10000",
    "type": " International Airport",
    "keyword": "Airport",
    "key": apiKey
}

result=getFromApi("/place/nearbysearch/json",queryParams,apiKey=apiKey)
airports_list=[]
for loc in result["results"]:
        airports_list.append({"Requirement":"Transport - Airports","name":loc["name"],"latitude":loc["geometry"]["location"]["lat"],"longitude":loc["geometry"]["location"]["lng"],"location":
                                    {"type":"Point",
                                     "coordinates":[loc["geometry"]["location"]["lng"],loc["geometry"]["location"]["lat"]]}})
airports_data=pd.DataFrame(airports_list)
airports_data.head()

200


,Requirement,name,latitude,longitude,location
0,Transport - Airports,King County International Airport,47.528166,-122.301826,"{'type': 'Point', 'coordinates': [-122.3018264..."
1,Transport - Airports,Signature Flight Support BFI - Boeing Field/Ki...,47.537634,-122.304032,"{'type': 'Point', 'coordinates': [-122.3040317..."
2,Transport - Airports,King County International Airport Airpark Hangars,47.520570,-122.298924,"{'type': 'Point', 'coordinates': [-122.2989238..."
3,Transport - Airports,Lake Union Seaplane Terminal,47.628547,-122.339838,"{'type': 'Point', 'coordinates': [-122.3398377..."
4,Transport - Airports,Airfield Business Center,47.521763,-122.299315,"{'type': 'Point', 'coordinates': [-122.2993152..."


In [31]:
transport_data=pd.concat([airports_data,metro_data,trainstation_data],axis=0)
transport_data.to_json("OUTPUT/transport_data.json",orient="records")

## 4º Requirement: 30% of the company have at least 1 child.

**Buscamos colegios en la zona:**

In [32]:
queryParams = {
    "location": "47.61186, -122.33581",
    "radius": "5000",
    "type": "Schools",
    "keyword":"Nursery School",
    "key": apiKey
}

result=getFromApi("/place/nearbysearch/json",queryParams,apiKey=apiKey)
Schools_locations=[]
for loc in result["results"]:
        Schools_locations.append({"Requirement":"Schools","name":loc["name"],"latitude":loc["geometry"]["location"]["lat"],"longitude":loc["geometry"]["location"]["lng"],"location":
                                    {"type":"Point",
                                     "coordinates":[loc["geometry"]["location"]["lng"],loc["geometry"]["location"]["lat"]]}})
schools_data=pd.DataFrame(Schools_locations)
schools_data.head()

200


,Requirement,name,latitude,longitude,location
0,Schools,Bright Horizons at Belltown,47.616922,-122.339203,"{'type': 'Point', 'coordinates': [-122.3392031..."
1,Schools,Bright Horizons at Seneca Street,47.612295,-122.319079,"{'type': 'Point', 'coordinates': [-122.3190792..."
2,Schools,mermaids lagoon,47.581009,-122.335738,"{'type': 'Point', 'coordinates': [-122.3357381..."
3,Schools,Illuminate School,47.641495,-122.320651,"{'type': 'Point', 'coordinates': [-122.3206511..."
4,Schools,The Andover School,47.580017,-122.385389,"{'type': 'Point', 'coordinates': [-122.385389,..."


In [33]:
schools_data.to_json("OUTPUT/schools_data.json",orient="records")

## 5º Requirement: Executives like Starbucks A LOT. Ensure there's a starbucks not to far.

**Buscamos Strabucks de la zona:**

In [34]:
queryParams = {
    "location": "47.61186, -122.33581",
    "radius": "5000",
    "type": "CoffeeShop",
    "keyword":"Starbucks",
    "key": apiKey
}

result=getFromApi("/place/nearbysearch/json",queryParams,apiKey=apiKey)
Starbucks_locations=[]
for loc in result["results"]:
    if loc["name"]=="Starbucks":
        Starbucks_locations.append({"Requirement":"Starbucks","name":loc["name"],"latitude":loc["geometry"]["location"]["lat"],"longitude":loc["geometry"]["location"]["lng"],"location":
                                    {"type":"Point",
                                     "coordinates":[loc["geometry"]["location"]["lng"],loc["geometry"]["location"]["lat"]]}})
Starbucks_data=pd.DataFrame(Starbucks_locations)
Starbucks_data.head()

200


,Requirement,name,latitude,longitude,location
0,Starbucks,Starbucks,47.610021,-122.342583,"{'type': 'Point', 'coordinates': [-122.342583,..."
1,Starbucks,Starbucks,47.607171,-122.309950,"{'type': 'Point', 'coordinates': [-122.30995, ..."
2,Starbucks,Starbucks,47.621039,-122.350774,"{'type': 'Point', 'coordinates': [-122.3507743..."
3,Starbucks,Starbucks,47.579085,-122.299620,"{'type': 'Point', 'coordinates': [-122.2996199..."
4,Starbucks,Starbucks,47.609009,-122.339942,"{'type': 'Point', 'coordinates': [-122.3399417..."


In [35]:
Starbucks_data.to_json("OUTPUT/starbucks_data.json",orient="records")

## 6º Requirement: All people in the company have between 25 and 40 years, give them some place to go to party.

**Buscamos pubs en la zona:**

In [36]:
queryParams = {
    "location": "47.61186, -122.33581",
    "radius": "5000",
    "type": "Lounge",
    "keyword":"Pub",
    "key": apiKey
}

result=getFromApi("/place/nearbysearch/json",queryParams,apiKey=apiKey)
pubs_locations=[]
for loc in result["results"]:
        pubs_locations.append({"Requirement":"Pubs","name":loc["name"],"latitude":loc["geometry"]["location"]["lat"],"longitude":loc["geometry"]["location"]["lng"],"location":
                                    {"type":"Point",
                                     "coordinates":[loc["geometry"]["location"]["lng"],loc["geometry"]["location"]["lat"]]}})
pubs_data=pd.DataFrame(pubs_locations)
pubs_data.head()

200


,Requirement,name,latitude,longitude,location
0,Pubs,The Pike Brewing Company,47.608232,-122.339743,"{'type': 'Point', 'coordinates': [-122.339743,..."
1,Pubs,Flatstick Pub - Pioneer Square,47.600143,-122.331002,"{'type': 'Point', 'coordinates': [-122.3310024..."
2,Pubs,Hopvine Pub,47.623362,-122.312815,"{'type': 'Point', 'coordinates': [-122.3128147..."
3,Pubs,McMenamins Six Arms,47.614108,-122.327767,"{'type': 'Point', 'coordinates': [-122.3277668..."
4,Pubs,Blarney Stone Pub & Restaurant,47.608279,-122.339295,"{'type': 'Point', 'coordinates': [-122.3392946..."


In [37]:
pubs_data.to_json("OUTPUT/pubs_data.json",orient="records")

**Juntamos todos los requirements en un mismo dataframe:**

In [38]:
requirements_data=pd.concat([millioncompanies_data,airports_data,metro_data,trainstation_data,Vegan_data,Starbucks_data,schools_data,pubs_data],axis=0)

In [39]:
requirements_data.head()

,Requirement,name,latitude,longitude,location
0,TechCompanies raised more 1 million,Wetpaint,47.603122,-122.333253,"{'type': 'Point', 'coordinates': [-122.333253,..."
0,TechCompanies raised more 1 million,Wetpaint,40.723731,-73.996431,"{'type': 'Point', 'coordinates': [-73.9964312,..."
2,TechCompanies raised more 1 million,Newsvine,47.618599,-122.358484,"{'type': 'Point', 'coordinates': [-122.358484,..."
3,TechCompanies raised more 1 million,iLike,47.615313,-122.323408,"{'type': 'Point', 'coordinates': [-122.323408,..."
4,TechCompanies raised more 1 million,PayScale,47.599934,-122.327696,"{'type': 'Point', 'coordinates': [-122.3276962..."


In [40]:
requirements_data["Requirement"].value_counts()

TechCompanies raised more 1 million    56
Vegan Restaurant                       30
Transport - Train                      30
Pubs                                   20
Schools                                20
Starbucks                              18
Transport - Airports                   13
Transport - Metro                      12
Name: Requirement, dtype: int64

In [41]:
requirements_data.to_json("OUTPUT/requirements_data.json",orient="records")

**Mapa con todos los requirements:**

https://public.tableau.com/profile/macarena6661#!/vizhome/RequirementsOfficesSeattleMap/Hoja1?publish=yes